In [44]:
from importlib import machinery

import matplotlib.pyplot as plt
from keras.layers import Input, Dense
from keras.models import Model
from keras.optimizers import RMSprop
from keras import backend as K
import tensorflow as tf

from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
import io
import pandas as pd

from sklearn.preprocessing import LabelEncoder


class MultiColumnLabelEncoder:

    def __init__(self, columns=None):
        self.columns = columns # array of column names to encode


    def fit(self, X, y=None):
        self.encoders = {}
        columns = X.columns if self.columns is None else self.columns
        for col in columns:
            self.encoders[col] = LabelEncoder().fit(X[col])
        return self


    def transform(self, X):
        output = X.copy()
        columns = X.columns if self.columns is None else self.columns
        for col in columns:
            output[col] = self.encoders[col].transform(X[col])
        return output


    def fit_transform(self, X, y=None):
        return self.fit(X,y).transform(X)


    def inverse_transform(self, X):
        output = X.copy()
        columns = X.columns if self.columns is None else self.columns
        for col in columns:
            output[col] = self.encoders[col].inverse_transform(X[col])
        return output

In [45]:
'''
다시 불러왔다.
'''

import pandas as pd
import pymysql

conn=pymysql.connect(host='database-1.cani4y1t2hfg.ap-northeast-2.rds.amazonaws.com',port=3306,user='slows14tem',password='!wjdgy0117',db='bigdata', cursorclass=pymysql.cursors.DictCursor)
cursor = conn.cursor()
sql = "select * from JSYoon_leadtime_preprocessed"
cursor.execute(sql)
result = cursor.fetchall()
conn.close()
train = pd.DataFrame(result)
train

,id,cheongguseobeonho,no,subject_,machinery,assembly,items,part1,Part2,key1,key2,leadtime,cheongguryang,gyeonjeok,gyeonjeoksuryang,gyeonjeokhwapye,gyeonjeokdanga,baljubeonho,baljucheo,balju,baljusuryang,baljugeumaek,baljugeumaek_currency,dt,miipgogigan,changgoipgo,changgoipgosuryang,control_no,ipgochanggo,changgochulgo,changgochulgosuryang,chulgoseonbak,chulgounbanseon,seonbagipgo,seonbagipgosuryang,wallyo_yeobu,발주-월,창고입고-발주
0,1,ELS-BS-ESP-2004002,12,NO.1 A/E C18 MAJOR O/H PARTS,NO.1 GENERATOR ENGINE,323-6480 LINES GP-FUEL,SEAL-O-RING-STOR,7.00E-275,None,7.00E-275,COOLER,194,1,2020-03-09,1,KRW,884530,BLO-BS-OSP-2003002,HAEIN Coporation_Cheonan,2020-03-09,1,884530,884530.0,40,None,2020-09-19,1,혜인 발주,BS,2020-09-19,1,BLO,본선 선적,0000-00-00,0,None,3,194 days
1,2,OCA-BS-ESP-2010004,5,운전시간 대비 #2 AUX ENG TOP END O/H PARTS,NO.2 GENERATOR ENGINE,GASKET KIT,OIL COOLER & LINES,7.00E-275,None,7.00E-275,COOLER,59,1,2020-07-21,1,KRW,905260,JBG-BS-OSP-2007002,HAEIN Coporation_Cheonan,2020-07-21,1,905260,905260.0,5,None,2020-09-18,1,혜인 발주,BS,2020-09-18,1,JBG,본선 선적,0000-00-00,0,None,7,59 days
2,3,MIR-BS-ESP-2011002,119,운전시간 대비 #2 AUX ENG(CAT 3512C) MAJOR O/H PARTS,NO.2 GENERATOR ENGINE,285-8374 MANIFOLD GP-EXH,WASHER,7.00E-275,None,7.00E-275,COOLER,61,1,2020-10-12,1,KRW,829820,BON-BS-OSP-2010002,HAEIN Coporation_Cheonan,2020-10-12,1,829820,829820.0,10,None,2020-12-12,1,None,BS,2020-12-12,1,BON,SANWA FONTAINE,0000-00-00,0,None,10,61 days
3,4,BLO-BS-ESP-2003002,16,#1 AUX ENG TOP END O/H PARTS,NO.1 GENERATOR ENGINE,159-8828 TURBO GP,BOLT-HIGH TEMP,7.00E-275,None,7.00E-275,COOLER,212,1,2020-11-16,1,KRW,829830,MIR-BS-OSP-2011001,HAEIN Coporation_Cheonan,2020-11-16,1,829830,829830.0,50,None,2021-06-16,1,혜인 발주,BS,2021-06-16,1,MIR,체항수리 시 사용,0000-00-00,0,None,11,212 days
4,5,DEO-BS-ESP-2105008,97,DEO-E-210512-01,NO.1 GENERATOR ENGINE,2N4727 INSTRUMNT PANEL GP,SEAL,7.00E-275,None,7.00E-275,COOLER,220,1,2021-01-15,1,KRW,829830,SEG-BS-OSP-2101010,HAEIN Coporation_Cheonan,2021-01-15,1,829830,829830.0,30,None,2021-08-23,1,None,BS,0000-00-00,0,None,None,0000-00-00,0,None,1,220 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17315,21080,BLO-BS-ESP-2006006,1,"BLO-F-ENGINE-200515-03 BOAT DAVIT CYL 외, AIRCO...",HYD. SYSTEM,5-753-2E_BOAT DAVIT & N.D CRANE LINE,HYD. CYLINDER_NET BOAT DAVIT,φ150 × φ85 × 695st,FF-SE-1CB150BB695ABD-YP2J,φ150 × φ85 × 695stFF-SE-1CB150BB695ABD-YP2J,CYLINDER,95,2,2020-06-11,2,JPY,455000,BLO-BS-OSP-2006003,"MARINE HYDROTEC CO.,LTD.",2020-06-11,2,10164700,109778760.0,75,None,2020-09-14,2,20-09-161,BW,0000-00-00,0,None,None,0000-00-00,0,None,6,95 days
17316,21081,BLO-BS-ESP-2004008,1,BLO-F-ENGINE-200403-01 (NAVI DECK CRANE DERRIN...,HYD. SYSTEM,5-753-2E_BOAT DAVIT & N.D CRANE LINE,HYD. CYLINDER_N.D CRANE_BOOM_LIFTER,φ160 × φ112 × 1130st,S-2CA160A20.6N1130-1,φ160 × φ112 × 1130stS-2CA160A20.6N1130-1,CYLINDER,67,1,2020-04-27,1,JPY,900000,BLO-BS-OSP-2004005,"MARINE HYDROTEC CO.,LTD.",2020-04-27,1,10350000,111780000.0,110,None,2020-07-03,1,20-07-114,BW,2020-08-26,1,BLO,BLO 상가시 선적,0000-00-00,0,None,4,67 days
17317,21082,ELS-BS-ESP-1908002,2,ELS-F-ENG-190628-02,6M NET BOAT,ELECTRIC CLUTCH,PULLY,Φ305X5V-3,None,Φ305X5V-3,BELT,25,1,2019-08-01,1,JPY,50000,ELS-BS-OSP-1908001,"PORT RELIEF ENGINEERING CO.,LTD.",2019-08-01,1,543000,5864400.0,5,None,2019-08-26,1,19-08-164,BW,2019-09-04,1,ELS,ONENESS,2019-10-02,1,완료,8,25 days
17318,21083,BLO-BS-ESP-1906014,2,BLO-F-ENGINE-190617-01 (NET BOAT PARTS),7M NET BOAT,HAND HYDRAULIC STEERING SYSTEM,CYLINDER,Φ50 x S195 CYLINDER,None,Φ50 x S195 CYLINDER,CYLINDER,25,1,2019-06-17,1,JPY,74250,BLO-BS-OSP-1906009,"PORT RELIEF ENGINEERING CO.,LTD.",2019-06-17,1,810068,8748734.4,5,None,2019-07-12,1,19-07-134,BW,2019-08-20,1,BLO,세이셀 컨테이너,2019-10-12,1,완료,6,25 days


In [46]:
features = ['machinery', 'assembly', 'items', 'part1', 'gyeonjeok', 'gyeonjeokhwapye', 'baljucheo', 'baljusuryang', 'baljugeumaek_currency', 'dt', 'key2']

In [16]:
# 조합 4개

import itertools
*combo, = itertools.combinations(train[features].columns, 4)
print(len(combo)) # 330개

for i in range(len(combo)):
    original_combo_repeat = train.loc[:, [combo[i][0], combo[i][1], combo[i][2], combo[i][3]]]
    original_combo_repeat = pd.concat([original_combo_repeat, train['leadtime']], axis=1)

    from pycaret.regression import *
    multi = MultiColumnLabelEncoder(columns=original_combo_repeat)
    X = multi.fit_transform(original_combo_repeat)
    features_combo_repeat = X.iloc[:, 0:4].columns

    from sklearn.model_selection import train_test_split
    x_train, x_test, y_train, y_test = train_test_split(X[features_combo_repeat], X['leadtime'])

    from sklearn.ensemble import RandomForestRegressor
    random_forest = RandomForestRegressor()
    model_rf = random_forest.fit(x_train, y_train)
    y_pred = random_forest.predict(x_test)
    y_pred_df = pd.DataFrame(y_pred)
    y_pred_df.columns = ['predicted']
    y_pred_df = y_pred_df.astype(int)

    TR = pd.concat([x_train[features_combo_repeat], y_train], axis=1)
    VAL = pd.concat([x_test[features_combo_repeat], y_test], axis=1)
    VAL2 = VAL.reset_index(drop=True)
    temp = pd.concat([y_pred_df, VAL2], axis=1)
    temp_inv = multi.inverse_transform(temp)
    temp_inv = temp_inv.reset_index(drop=True)

    from sklearn.metrics import mean_squared_error, mean_squared_log_error, mean_absolute_error
    official_y_true = temp_inv['leadtime']
    official_y_pred = temp_inv['predicted']

    print(i, end=' ')
    if (mean_squared_error(official_y_true, official_y_pred)**0.5) < 23:
        print(features_combo_repeat)
        print('**rmse    ', mean_squared_error(official_y_true, official_y_pred)**0.5)
        print('**msle   ', mean_squared_log_error(official_y_true, official_y_pred))
        print('**absolute   ', mean_absolute_error(official_y_true, official_y_pred))

330
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 Index(['assembly', 'gyeonjeok', 'gyeonjeokhwapye', 'dt'], dtype='object')
**rmse     22.71133747458583
**msle    0.09442783897628615
**absolute    8.772517321016167
173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 2

KeyboardInterrupt: 

In [47]:
features_4 = ['assembly', 'gyeonjeok', 'gyeonjeokhwapye', 'dt']
temp4 = train[features_4]
needs_to_be_encoded = ['assembly', 'gyeonjeok', 'gyeonjeokhwapye']
multi = MultiColumnLabelEncoder(columns=needs_to_be_encoded)
X = multi.fit_transform(temp4)
X = pd.concat([X, train['leadtime']], axis=1)
X

,assembly,gyeonjeok,gyeonjeokhwapye,dt,leadtime
0,272,245,2,40,194
1,1129,314,2,5,59
2,199,351,2,10,61
3,67,369,2,50,212
4,230,405,2,30,220
...,...,...,...,...,...
17315,457,294,1,75,95
17316,457,270,1,110,67
17317,861,125,1,5,25
17318,1154,98,1,5,25


In [49]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X[features_4], X['leadtime'])
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor
random_forest = RandomForestRegressor()
model4 = random_forest.fit(x_train, y_train)
y_pred = random_forest.predict(x_test)
y_pred_df = pd.DataFrame(y_pred)
y_pred_df.columns = ['leadtime_predicted']
y_pred_df = y_pred_df.astype(int)
TR = pd.concat([x_train[features_4], y_train], axis=1)
VAL = pd.concat([x_test[features_4], y_test], axis=1)
VAL2 = VAL.reset_index(drop=True)
temp = pd.concat([y_pred_df, VAL2], axis=1)
temp_inv = multi.inverse_transform(temp)
temp_inv['차이'] = (temp_inv['leadtime_predicted'] - temp_inv['leadtime']).abs()
temp_inv.drop(temp_inv[(temp_inv['차이'] >= 4)].index, inplace=True)
temp_inv = temp_inv.reset_index(drop=True)
temp_inv.loc[:, ['leadtime_predicted', 'assembly', 'leadtime', '차이']]

,leadtime_predicted,assembly,leadtime,차이
0,3,242-6277 LINES GP-ENG OIL,3,0
1,34,MACHINE BOTTOM PART,34,0
2,18,FIG 64. STARTER MOTOR,18,0
3,146,FUEL VAVLE,146,0
4,124,282-4202 LINES GP-TURBO O,124,0
...,...,...,...,...
3274,55,PISTON AND CONNECTING ROD ASSEMBLY AND SAFETY ...,56,1
3275,71,OUTPUT GROUP,71,0
3276,45,CRANKSHAFT AND FRONT BEARING COVER REPLACEMENT...,45,0
3277,14,141-9284 LINES GP-WATER,14,0


In [52]:
from sklearn.metrics import mean_squared_error, mean_squared_log_error, mean_absolute_error

official_y_true = temp_inv['leadtime']
official_y_pred = temp_inv['leadtime_predicted']
# RMSE, MSLE, MAE
print('**rmse    ', mean_squared_error(official_y_true, official_y_pred)**0.5)
print('**msle   ', mean_squared_log_error(official_y_true, official_y_pred))
print('**absolute   ', mean_absolute_error(official_y_true, official_y_pred))

**rmse     0.7889523861695166
**msle    0.002396536784820423
**absolute    0.3199146081122293


In [39]:
import joblib

joblib.dump(model4, './model4.pkl')

['./model4.pkl']

In [13]:
# 조합 5개
import itertools
*combo, = itertools.combinations(train[features].columns, 5)
print(len(combo))

for i in range(len(combo)):
    # 여기랑
    original_combo_repeat = train.loc[:, [combo[i][0], combo[i][1], combo[i][2], combo[i][3], combo[i][4]]]
    original_combo_repeat = pd.concat([original_combo_repeat, train['leadtime']], axis=1)
    from pycaret.regression import *
    multi = MultiColumnLabelEncoder(columns=original_combo_repeat)
    X = multi.fit_transform(original_combo_repeat)
    # 여기를 바꿔줘야 됨
    features_combo_repeat = X.iloc[:, 0:5].columns
    from sklearn.model_selection import train_test_split
    x_train, x_test, y_train, y_test = train_test_split(X[features_combo_repeat], X['leadtime'])
    from sklearn.ensemble import RandomForestRegressor
    random_forest = RandomForestRegressor()
    model_rf = random_forest.fit(x_train, y_train)
    y_pred = random_forest.predict(x_test)
    y_pred_df = pd.DataFrame(y_pred)
    y_pred_df.columns = ['predicted']
    y_pred_df = y_pred_df.astype(int)
    TR = pd.concat([x_train[features_combo_repeat], y_train], axis=1)
    VAL = pd.concat([x_test[features_combo_repeat], y_test], axis=1)
    VAL2 = VAL.reset_index(drop=True)
    temp = pd.concat([y_pred_df, VAL2], axis=1)
    temp_inv = multi.inverse_transform(temp)
    # temp_inv['차이'] = (temp_inv['predicted'] - temp_inv['leadtime']).abs()
    temp_inv = temp_inv.reset_index(drop=True)
    from sklearn.metrics import mean_squared_error, mean_squared_log_error, mean_absolute_error
    official_y_true = temp_inv['leadtime']
    official_y_pred = temp_inv['predicted']
    '''
    feature를 다 때려넣고 만든 rmse가 57이었다.
    그것보다 더 낮은 mse값이 존재할까?
    현재 rmse 23
    '''
    print(i, end=' ')
    if (mean_squared_error(official_y_true, official_y_pred)**0.5) < 21:
        print(features_combo_repeat)
        print('**rmse    ', mean_squared_error(official_y_true, official_y_pred)**0.5)
        print('**msle   ', mean_squared_log_error(official_y_true, official_y_pred))
        print('**absolute   ', mean_absolute_error(official_y_true, official_y_pred))

462
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26


KeyboardInterrupt: 

In [40]:
# 조합 6개
import itertools
*combo, = itertools.combinations(train[features].columns, 6)
print(len(combo))

for i in range(len(combo)):
    # 여기랑
    original_combo_repeat = train.loc[:, [combo[i][0], combo[i][1], combo[i][2], combo[i][3], combo[i][4], combo[i][5]]]
    original_combo_repeat = pd.concat([original_combo_repeat, train['leadtime']], axis=1)
    from pycaret.regression import *
    multi = MultiColumnLabelEncoder(columns=original_combo_repeat)
    X = multi.fit_transform(original_combo_repeat)
    # 여기를 바꿔줘야 됨
    features_combo_repeat = X.iloc[:, 0:6].columns
    from sklearn.model_selection import train_test_split
    x_train, x_test, y_train, y_test = train_test_split(X[features_combo_repeat], X['leadtime'])
    from sklearn.ensemble import RandomForestRegressor
    random_forest = RandomForestRegressor()
    model_rf = random_forest.fit(x_train, y_train)
    y_pred = random_forest.predict(x_test)
    y_pred_df = pd.DataFrame(y_pred)
    y_pred_df.columns = ['predicted']
    y_pred_df = y_pred_df.astype(int)
    TR = pd.concat([x_train[features_combo_repeat], y_train], axis=1)
    VAL = pd.concat([x_test[features_combo_repeat], y_test], axis=1)
    VAL2 = VAL.reset_index(drop=True)
    temp = pd.concat([y_pred_df, VAL2], axis=1)
    temp_inv = multi.inverse_transform(temp)
    # temp_inv['차이'] = (temp_inv['predicted'] - temp_inv['leadtime']).abs()
    temp_inv = temp_inv.reset_index(drop=True)
    from sklearn.metrics import mean_squared_error, mean_squared_log_error, mean_absolute_error
    official_y_true = temp_inv['leadtime']
    official_y_pred = temp_inv['predicted']
    '''
    feature를 다 때려넣고 만든 rmse가 57이었다.
    그것보다 더 낮은 mse값이 존재할까?
    현재 rmse 22
    '''
    print(i)
    if (mean_squared_error(official_y_true, official_y_pred)**0.5) < 21:
        print(features_combo_repeat)
        print('**rmse    ', mean_squared_error(official_y_true, official_y_pred)**0.5)
        print('**msle   ', mean_squared_log_error(official_y_true, official_y_pred))
        print('**absolute   ', mean_absolute_error(official_y_true, official_y_pred))

462
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
27

In [ ]:
# 조합 7개
import itertools
*combo, = itertools.combinations(train[features].columns, 7)
print(len(combo))

for i in range(len(combo)):
    # 여기랑
    original_combo_repeat = train.loc[:, [combo[i][0], combo[i][1], combo[i][2], combo[i][3], combo[i][4], combo[i][5], combo[i][6]]]
    original_combo_repeat = pd.concat([original_combo_repeat, train['leadtime']], axis=1)
    from pycaret.regression import *
    multi = MultiColumnLabelEncoder(columns=original_combo_repeat)
    X = multi.fit_transform(original_combo_repeat)
    # 여기를 바꿔줘야 됨
    features_combo_repeat = X.iloc[:, 0:7].columns
    from sklearn.model_selection import train_test_split
    x_train, x_test, y_train, y_test = train_test_split(X[features_combo_repeat], X['leadtime'])
    from sklearn.ensemble import RandomForestRegressor
    random_forest = RandomForestRegressor()
    model_rf = random_forest.fit(x_train, y_train)
    y_pred = random_forest.predict(x_test)
    y_pred_df = pd.DataFrame(y_pred)
    y_pred_df.columns = ['predicted']
    y_pred_df = y_pred_df.astype(int)
    TR = pd.concat([x_train[features_combo_repeat], y_train], axis=1)
    VAL = pd.concat([x_test[features_combo_repeat], y_test], axis=1)
    VAL2 = VAL.reset_index(drop=True)
    temp = pd.concat([y_pred_df, VAL2], axis=1)
    temp_inv = multi.inverse_transform(temp)
    # temp_inv['차이'] = (temp_inv['predicted'] - temp_inv['leadtime']).abs()
    temp_inv = temp_inv.reset_index(drop=True)
    from sklearn.metrics import mean_squared_error, mean_squared_log_error, mean_absolute_error
    official_y_true = temp_inv['leadtime']
    official_y_pred = temp_inv['predicted']
    '''
    feature를 다 때려넣고 만든 rmse가 57이었다.
    그것보다 더 낮은 mse값이 존재할까?
    현재 rmse 23
    '''
    print(i)
    if (mean_squared_error(official_y_true, official_y_pred)**0.5) < 21:
        print(features_combo_repeat)
        print('**rmse    ', mean_squared_error(official_y_true, official_y_pred)**0.5)
        print('**msle   ', mean_squared_log_error(official_y_true, official_y_pred))
        print('**absolute   ', mean_absolute_error(official_y_true, official_y_pred))